In [1]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=10000, random_state=42)

In [12]:
import time
from sklearn.ensemble import RandomForestClassifier

t1 = time.time()
rfc = RandomForestClassifier(n_jobs=-1, random_state=42)
rfc.fit(X_train, y_train)
t0 = time.time()
print('Training took {:.2f}s'.format(t0-t1))

Training took 14.13s


In [13]:
from sklearn.metrics import accuracy_score

y_pred = rfc.predict(X_test)
accuracy_score(y_test, y_pred)

0.9674

In [14]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95, random_state=42)
X_train_reduced = pca.fit_transform(X_train)

In [15]:
t1 = time.time()
rfc.fit(X_train_reduced,y_train)
t0 = time.time()
print('Training took {:.2f}s'.format(t0-t1))

Training took 23.27s


In [17]:
X_test_reduced = pca.transform(X_test)
y_pred = rfc.predict(X_test_reduced)
accuracy_score(y_test, y_pred)

0.9469

PCA sucked for this model, it increased training time and severely crippled performance